# Pandas Medallion Architecture Learning Guide

Welcome to your Pandas learning repository! This notebook will guide you through the core concepts of data processing using pandas, focusing on the Medallion Architecture (Bronze, Silver, Gold layers). You'll learn how to ingest raw data, transform it, and aggregate it for analytical purposes using pandas - a great foundation before moving to distributed processing with PySpark.

## What is the Medallion Architecture?

The Medallion Architecture is a data design pattern used to logically organize data in a data lake:
- **Bronze Layer**: Raw data as-is from source systems
- **Silver Layer**: Cleaned, validated, and enriched data
- **Gold Layer**: Business-level aggregations for analytics and reporting


## 1. Setting up your pandas environment

Let's start by importing the necessary libraries and checking our environment.

**Exercise 1.1**: Import pandas and other necessary libraries.


In [7]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Current working directory: {os.getcwd()}")

Pandas version: 2.3.1
NumPy version: 2.3.1
Current working directory: /Users/vamsi_mbmax/Developer/VAM_Documents/01_vam_PROJECTS/LEARNING/proj_Databases/dev_proj_Databases/practise_db_book_pyspark_learn/notebooks


## 2. Bronze Layer: Raw Data Ingestion

The Bronze layer is where raw data is ingested as-is from source systems. We'll read the sample sales data from `data/raw/sales_data.csv` and save it to the bronze layer.

**Exercise 2.1**: Read the raw CSV file and explore its structure.


In [8]:
# Read raw data from CSV
bronze_df = pd.read_csv('../data/raw/sales_data.csv')

print("Bronze Layer (Raw Data) Schema:")
print(bronze_df.dtypes)
print(f"\nDataset shape: {bronze_df.shape}")
print(f"Total records: {len(bronze_df)}")

print("\nFirst 5 rows:")
display(bronze_df.head())

print("\nBasic statistics:")
display(bronze_df.describe())


Bronze Layer (Raw Data) Schema:
transaction_id        int64
product_name         object
customer_name        object
city                 object
quantity              int64
price               float64
transaction_date     object
dtype: object

Dataset shape: (1000, 7)
Total records: 1000

First 5 rows:


,transaction_id,product_name,customer_name,city,quantity,price,transaction_date
0,1,Keyboard,Customer_49,New York,4,678.03,2024-06-30
1,2,Laptop,Customer_37,Los Angeles,3,637.83,2024-07-05
2,3,Laptop,Customer_40,Houston,2,665.58,2024-10-21
3,4,Headphones,Customer_3,Los Angeles,4,222.11,2024-11-20
4,5,Mouse,Customer_10,Los Angeles,3,87.20,2024-07-25



Basic statistics:


,transaction_id,quantity,price
count,1000.000000,1000.000000,1000.000000
mean,500.500000,2.508000,495.977220
std,288.819436,1.131024,281.025972
min,1.000000,1.000000,11.160000
25%,250.750000,1.000000,259.810000
50%,500.500000,3.000000,489.015000
75%,750.250000,4.000000,742.457500
max,1000.000000,4.000000,999.910000


**Exercise 2.2**: Save the bronze layer data to parquet format for better performance.


In [9]:
# Create bronze directory if it doesn't exist
os.makedirs('../data/bronze', exist_ok=True)

# Save bronze data as parquet (more efficient than CSV)
bronze_df.to_parquet('../data/bronze/sales.parquet', index=False, engine="pyarrow")

print("✅ Bronze layer data saved to '../data/bronze/sales.parquet'")
print(f"File size: {os.path.getsize('../data/bronze/sales.parquet')} bytes")


✅ Bronze layer data saved to '../data/bronze/sales.parquet'
File size: 20922 bytes


## 3. Silver Layer: Cleaned and Conformed Data

The Silver layer contains cleaned, conformed, and enriched data. We'll perform data quality checks, calculate derived columns, and add metadata.

**Exercise 3.1**: Load bronze data and perform data quality checks.


In [10]:
# Load bronze data
bronze_df = pd.read_parquet('../data/bronze/sales.parquet')

print("Data Quality Checks:")
print(f"Missing values:\n{bronze_df.isnull().sum()}")
print(f"\nDuplicate rows: {bronze_df.duplicated().sum()}")
print(f"\nUnique values:")
for col in bronze_df.columns:
    print(f"  {col}: {bronze_df[col].nunique()}")


Data Quality Checks:
Missing values:
transaction_id      0
product_name        0
customer_name       0
city                0
quantity            0
price               0
transaction_date    0
dtype: int64

Duplicate rows: 0

Unique values:
  transaction_id: 1000
  product_name: 6
  customer_name: 50
  city: 5
  quantity: 4
  price: 997
  transaction_date: 347


**Exercise 3.2**: Create the silver layer with enrichments and transformations.


In [16]:
# Create silver layer with transformations
silver_df = bronze_df.copy()

# Convert transaction_date to datetime
silver_df['transaction_date'] = pd.to_datetime(silver_df['transaction_date'])

# Calculate total_price (derived column)
silver_df['total_price'] = silver_df['quantity'] * silver_df['price']

# Add processing metadata
silver_df['processing_timestamp'] = datetime.now()
silver_df['data_source'] = 'sales_csv'

# Extract date components for easier analysis
silver_df['year'] = silver_df['transaction_date'].dt.year
silver_df['month'] = silver_df['transaction_date'].dt.month
silver_df['quarter'] = silver_df['transaction_date'].dt.quarter
silver_df['day_of_week'] = silver_df['transaction_date'].dt.day_name()

# Add business logic columns
silver_df['price_category'] = pd.cut(silver_df['price'],
                                    bins=[0, 100, 500, 1000, float('inf')],
                                    labels=['Low', 'Medium', 'High', 'Premium'])

silver_df['order_size'] = pd.cut(silver_df['quantity'],
                                bins=[0, 1, 3, 5, float('inf')],
                                labels=['Single', 'Small', 'Medium', 'Large'])

print("Silver Layer (Cleaned/Enriched) Schema:")
print(silver_df.dtypes)
print(f"\nDataset shape: {silver_df.shape}")

print("\nSample enriched data:")
display(silver_df.head())


Silver Layer (Cleaned/Enriched) Schema:
transaction_id                   int64
product_name                    object
customer_name                   object
city                            object
quantity                         int64
price                          float64
transaction_date        datetime64[ns]
total_price                    float64
processing_timestamp    datetime64[us]
data_source                     object
year                             int32
month                            int32
quarter                          int32
day_of_week                     object
price_category                category
order_size                    category
dtype: object

Dataset shape: (1000, 16)

Sample enriched data:


,transaction_id,product_name,customer_name,city,quantity,price,transaction_date,total_price,processing_timestamp,data_source,year,month,quarter,day_of_week,price_category,order_size
0,1,Keyboard,Customer_49,New York,4,678.03,2024-06-30,2712.12,2025-07-17 14:55:48.298087,sales_csv,2024,6,2,Sunday,High,Medium
1,2,Laptop,Customer_37,Los Angeles,3,637.83,2024-07-05,1913.49,2025-07-17 14:55:48.298087,sales_csv,2024,7,3,Friday,High,Small
2,3,Laptop,Customer_40,Houston,2,665.58,2024-10-21,1331.16,2025-07-17 14:55:48.298087,sales_csv,2024,10,4,Monday,High,Small
3,4,Headphones,Customer_3,Los Angeles,4,222.11,2024-11-20,888.44,2025-07-17 14:55:48.298087,sales_csv,2024,11,4,Wednesday,Medium,Medium
4,5,Mouse,Customer_10,Los Angeles,3,87.20,2024-07-25,261.60,2025-07-17 14:55:48.298087,sales_csv,2024,7,3,Thursday,Low,Small


**Exercise 3.3**: Apply data quality filters and save the silver layer.


In [7]:
# Apply business rules and filters
print(f"Records before filtering: {len(silver_df)}")

# Filter out invalid data
silver_filtered = silver_df[
    (silver_df['quantity'] > 0) &
    (silver_df['price'] > 0) &
    (silver_df['total_price'] > 0)
]

print(f"Records after filtering: {len(silver_filtered)}")

# Save silver layer
os.makedirs('../data/silver', exist_ok=True)
silver_filtered.to_parquet('../data/silver/sales.parquet', index=False)
print("\n✅ Silver layer data saved to '../data/silver/sales.parquet'")


Records before filtering: 1000
Records after filtering: 1000

✅ Silver layer data saved to '../data/silver/sales.parquet'


## 4. Gold Layer: Aggregated Data for Analytics

The Gold layer is optimized for analytics and reporting. We'll create various aggregations suitable for business intelligence and reporting.

**Exercise 4.1**: Create sales summary aggregation by product and city.


In [17]:
# Load silver data
silver_df = pd.read_parquet('../data/silver/sales.parquet')

# Create gold layer aggregation: Sales summary by product and city
gold_sales_summary = silver_df.groupby(['product_name', 'city']).agg({
    'transaction_id': 'count',
    'total_price': ['sum', 'mean'],
    'quantity': ['sum', 'mean']
}).round(2)

# Flatten column names
gold_sales_summary.columns = [
    'total_transactions', 'total_revenue', 'avg_order_value',
    'total_quantity_sold', 'avg_quantity_per_order'
]

gold_sales_summary = gold_sales_summary.reset_index()

print("Gold Layer - Sales Summary Schema:")
print(gold_sales_summary.dtypes)
print(f"\nDataset shape: {gold_sales_summary.shape}")

print("\nSample aggregated data:")
display(gold_sales_summary.head())


Gold Layer - Sales Summary Schema:
product_name               object
city                       object
total_transactions          int64
total_revenue             float64
avg_order_value           float64
total_quantity_sold         int64
avg_quantity_per_order    float64
dtype: object

Dataset shape: (30, 7)

Sample aggregated data:


,product_name,city,total_transactions,total_revenue,avg_order_value,total_quantity_sold,avg_quantity_per_order
0,Headphones,Chicago,37,48471.14,1310.03,91,2.46
1,Headphones,Houston,29,30021.19,1035.21,75,2.59
2,Headphones,Los Angeles,34,39736.61,1168.72,86,2.53
3,Headphones,New York,33,33105.74,1003.20,71,2.15
4,Headphones,Phoenix,36,49107.78,1364.10,79,2.19


**Exercise 4.2**: Perform analytics queries on the Gold table.


In [9]:
# Analytics queries on Gold layer
print("=== ANALYTICS INSIGHTS ===")

# 1. Top 5 products by revenue
print("\n1. Top 5 products by total revenue:")
top_products = gold_sales_summary.groupby('product_name')['total_revenue'].sum().sort_values(ascending=False).head(5)
display(top_products)

# 2. Top 5 cities by revenue
print("\n2. Top 5 cities by total revenue:")
top_cities = gold_sales_summary.groupby('city')['total_revenue'].sum().sort_values(ascending=False).head(5)
display(top_cities)

# 3. Best performing product-city combinations
print("\n3. Top 10 product-city combinations by revenue:")
top_combinations = gold_sales_summary.sort_values('total_revenue', ascending=False).head(10)
display(top_combinations[['product_name', 'city', 'total_revenue', 'total_transactions']])

# Save gold layer
os.makedirs('../data/gold', exist_ok=True)
gold_sales_summary.to_parquet('../data/gold/sales_summary.parquet', index=False)
print("\n✅ Gold layer data saved to '../data/gold/sales_summary.parquet'")


=== ANALYTICS INSIGHTS ===

1. Top 5 products by total revenue:


product_name
Keyboard      222673.37
Webcam        215183.29
Monitor       211023.32
Headphones    200442.46
Mouse         196873.03
Name: total_revenue, dtype: float64


2. Top 5 cities by total revenue:


city
New York       281638.00
Los Angeles    268921.61
Houston        239040.89
Chicago        229543.11
Phoenix        220071.73
Name: total_revenue, dtype: float64


3. Top 10 product-city combinations by revenue:


,product_name,city,total_revenue,total_transactions
28,Webcam,New York,56749.55,37
18,Monitor,New York,54242.34,36
7,Keyboard,Los Angeles,53440.50,40
23,Mouse,New York,52080.40,38
8,Keyboard,New York,51416.59,40
4,Headphones,Phoenix,49107.78,36
0,Headphones,Chicago,48471.14,37
27,Webcam,Los Angeles,46329.47,39
12,Laptop,Los Angeles,45598.43,35
6,Keyboard,Houston,45500.22,38



✅ Gold layer data saved to '../data/gold/sales_summary.parquet'


## 5. Summary and Cleanup

Let's summarize what we've accomplished in our Medallion Architecture implementation.


In [10]:
print("=" * 60)
print("MEDALLION ARCHITECTURE PIPELINE COMPLETED!")
print("=" * 60)
print("✅ Bronze Layer: Raw data ingested from CSV to Parquet")
print("✅ Silver Layer: Data cleaned, enriched with calculated columns")
print("✅ Gold Layer: Data aggregated for analytics and reporting")
print("✅ Analytics: Performed comprehensive data analysis")
print("=" * 60)

# File summary
print("\nFiles created:")
data_dirs = ['../data/bronze', '../data/silver', '../data/gold']
for dir_path in data_dirs:
    if os.path.exists(dir_path):
        files = os.listdir(dir_path)
        print(f"  {dir_path}: {files}")

print("\n🎉 Congratulations! You've successfully implemented a Medallion Architecture using pandas!")
print("Next step: Try the PySpark version for distributed data processing!")


MEDALLION ARCHITECTURE PIPELINE COMPLETED!
✅ Bronze Layer: Raw data ingested from CSV to Parquet
✅ Silver Layer: Data cleaned, enriched with calculated columns
✅ Gold Layer: Data aggregated for analytics and reporting
✅ Analytics: Performed comprehensive data analysis

Files created:
  ../data/bronze: ['sales.parquet']
  ../data/silver: ['sales.parquet']
  ../data/gold: ['sales_summary.parquet']

🎉 Congratulations! You've successfully implemented a Medallion Architecture using pandas!
Next step: Try the PySpark version for distributed data processing!


## Next Steps

Now that you've mastered the Medallion Architecture with pandas, here are some suggestions for further learning:

### Immediate Next Steps:
- **Try the PySpark version**: Move to `practice_pyspark_medallion_architecture_guide.ipynb` for distributed processing
- **Experiment with different data sources**: JSON, Excel, databases
- **Add more complex transformations**: Pivot tables, time series analysis, statistical functions

### Advanced Concepts to Explore:
- **Data Quality Monitoring**: Implement data validation rules and monitoring
- **Incremental Processing**: Handle new data arriving daily/hourly
- **Schema Evolution**: Handle changes in data structure over time
- **Performance Optimization**: Chunking, parallel processing, memory management

### Production Considerations:
- **Error Handling**: Robust error handling and logging
- **Configuration Management**: External config files for flexibility
- **Monitoring and Alerting**: Track pipeline health and performance
- **Testing**: Unit tests and data quality tests

Happy data engineering! 🚀
